## Inconsistent MCCs
Let us look at the following simple example of inconsistent MCCs. Assume there are three trees $T_1$, $T_2$ and $T_3$. Let $T_2$ and $T_3$ have the same topology. Any of the three branches could be removed to make the trees compatible. Which branch is chosen should be based on branch length, however as branch length is determined by the number of mutations on a branch (and potentially other external information) it is possible that between trees $T_1$ and $T_2$ branch A will be removed, but between trees $T_1$ and $T_3$ branch C will be removed. This would lead to the following MCCs, which are inconsistent.

* T_1 and T_2: {A}, {B, C}
* T_1 and T_3: {C}, {A, B}
* T_2 and T_3: {A, B, C}

Note that this is an issue that can only be fixed by finding MCCs simultaneously when using TreeKnit

In [11]:
using TreeTools
T_1 = "(A:3,(B:2,C:2):1)R;"
T_2 = "((A:3,B:3):1,C:4)R;"
T_3 = "((A:3,B:3):1,C:4)R;"
t1 = node2tree(TreeTools.parse_newick(T_1))
t2 = node2tree(TreeTools.parse_newick(T_2))
t3 = node2tree(TreeTools.parse_newick(T_3))
print("Tree 1")
TreeTools.print_tree_ascii("", t1)
print("Tree 2")
TreeTools.print_tree_ascii("", t2)
print("Tree 3")
TreeTools.print_tree_ascii("", t3)

Tree 1
  ____________________________________________________________________________ A
_|
 |                         ___________________________________________________ B
 |________________________|
                          |___________________________________________________ C
Tree 2
                     _________________________________________________________ A
  __________________|
_|                  |_________________________________________________________ B
 |
 |____________________________________________________________________________ C
Tree 3
                     _________________________________________________________ A
  __________________|
_|                  |_________________________________________________________ B
 |
 |____________________________________________________________________________ C


## Order Effects MCCs

1. This is a simple example illustrating the common effect of changing the order of evaluating trees and resolving polytomies. It should be noted that the total number of recombination events between all tree pairs stays the same, however this number changes between individual tree pairs. The order of evaluation leads to recombination events being found more in later evaluated tree pairs, leading to different local results.

In [14]:
T_1 = "(((A:1,B:1):1,C:2):1,D:3,E:3)R;"
T_2 = "(((A:1,B:1):1,C:2):1,(D:2,E:2):1)R;"
T_3 = "(((A:1,C:1):1,D:2):1,(B:2,E:2):1)R;"

print("Tree 1")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_1)))
print("Tree 2")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_2)))
print("Tree 3")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_3)))

Tree 1
                                                    _________________________ A
                           ________________________|
  ________________________|                        |_________________________ B
 |                        |
 |                        |__________________________________________________ C
_|
 |___________________________________________________________________________ D
 |
 |___________________________________________________________________________ E
Tree 2
                                                    _________________________ A
                           ________________________|
  ________________________|                        |_________________________ B
 |                        |
_|                        |__________________________________________________ C
 |
 |                         __________________________________________________ D
 |________________________|
                          |________________________________________

### Order: 1, 2, 3

* tree 1 and tree 2: resolve (D,E) {A, B, C, D, E}
* tree 1 and tree 2: at least 2 recombination events, e.g. {B}, {D}, {A, C, E}
* tree 2 and tree 3: same possible recombination events as between tree 2 and tree 1 e.g. {B}, {D}, {A, C, E}

### Order: 1, 3, 2

* tree 1 and tree 3: resolve ((A,B),C),D), {B}, {A, C, D, E}
* tree 1 and tree 2: at least 1 recombination event e.g. {B}, {A, C, D, E}
* tree 2 and tree 3: at least 2 recombination events, e.g. {B}, {D}, {A, C, E}

2. It is also possible that the order tree pairs are evaluated in lead to the inference of more or less recombination events. I was only able to find examples of this when four trees are evaluated, but this is probably also possible for more complex three tree cases. For this example I created the true underlying trees before removing internal branches to create polytomies.

In [18]:
true_T_1 = "((((A,B),C),D),((E,F),(G,H)))R;"
true_T_2 = "(((A,B),(C,D)),((E,F),(G,H)))R;"
true_T_3 = "(((A,B),(C,D)),(((E,G),F),H))R;"
true_T_4 = true_T_2

T_1 = "((((A,B),C),D),(E,F,G,H))R;"
T_2 = "(((A,B),C,D),((E,F),(G,H)))R;"
T_3 = "((A,B,C,D),(((E,G),F),H))R;"
T_4 = true_T_2

print("True Tree 1")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(true_T_1)))
print("Input Tree 1")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_1)))
print("True Tree 2")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(true_T_2)))
print("Input Tree 2")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_2)))
print("True Tree 3")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(true_T_3)))
print("Input Tree 3")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_3)))
print("True Tree 4")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(true_T_4)))
print("Input Tree 4")
TreeTools.print_tree_ascii("", node2tree(TreeTools.parse_newick(T_4)))

True Tree 1
 not all branch lengths known, assuming identical branch lengths

                                                          ___________________ A
                                        _________________|
                     __________________|                 |___________________ B
                    |                  |
  __________________|                  |__________________ C
 |                  |
 |                  |___________________ D
_|
 |                                      __________________ E
 |                   __________________|
 |                  |                  |__________________ F
 |__________________|
                    |                   __________________ G
                    |__________________|
                                       |__________________ H
Input Tree 1
 not all branch lengths known, assuming identical branch lengths

                                                          ___________________ A
                          

### True MCCs

* tree 1 and tree 2: {C}, {A, B, D, E, F, G, H}
* tree 1 and tree 3: {C}, {G}, {A, B, D, E, F, H}
* tree 1 and tree 4: {C}, {A, B, D, E, F, G, H}
* tree 2 and tree 3: {G}, {A, B, C, D, E, F, H}
* tree 2 and tree 4: {A, B, C, D, E, F, G, H}
* tree 3 and tree 4: {G}, {A, B, C, D, E, F, H}

There are a total of 6 recombination events

### Order: 1, 3, 2, 4

* tree 1 and tree 2: {G}, {A, B, C, D, E, F, H}
* tree 1 and tree 3: {A, B, C, D, E, F, G, H}
* tree 1 and tree 4: {C}, {G}, {A, B, D, E, F, H}
* tree 2 and tree 3: {G}, {A, B, C, D, E, F, H}
* tree 2 and tree 4: {C}, {G}, {A, B, D, E, F, H}
* tree 3 and tree 4: {C}, {A, B, D, E, F, G, H}

Not only do we infer the wrong pair wise recombination events there are a total of 7 recombination events instead of 6.

### Order: 4, 2, 1, 3

This order will return the true recombination events